In [ ]:
#@title Install dependencies
!pip install google-adk

In [ ]:
#@title Imports

from google.adk.planners import BuiltInPlanner
from google.adk.agents.llm_agent import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.planners import BuiltInPlanner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import ThinkingConfig


import os
import time
import random
import uuid #

In [ ]:
#@title API Setup

from getpass import getpass



GEMINI=getpass("Enter your GEMINI API KEY: ")
os.environ["GOOGLE_API_KEY"]=GEMINI
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

model= "gemini-2.5-flash"

Enter your GEMINI API KEY: ··········
Google API Key set: Yes


In [ ]:
#@title Tools Definition
# --- Tool 1: Document Validation ---
def validate_document(document_ids: list[str]) -> dict:
    """
    Validates if the required application documents are present and complete.
    Use this first to ensure the application is ready for processing.
    Returns a status of 'validated' or 'incomplete'.
    """
    print("--- Tool Called: validate_document() ---")
    time.sleep(1)
    if not document_ids or len(document_ids) < 2:
        return {"status": "incomplete", "missing_docs": ["income_proof", "id_proof"]}
    return {"status": "validated"}

validate_document_tool = FunctionTool(func=validate_document)


# --- Tool 2: Credit Check ---
def run_credit_check(borrower_id: str) -> dict:
    """
    Retrieves a borrower's credit score by calling the credit bureau API.
    This should be done after documents are validated.
    """
    print(f"--- Tool Called: run_credit_check(borrower_id='{borrower_id}') ---")
    time.sleep(2)
    if borrower_id == "Borrower-400":
      score = 450
      report_summary = "Credit history is compromised."
    else:
      score = random.randint(750, 850) # Simulate a good credit score
      report_summary = "Credit history is clean."
    return {"credit_score": score, "report_summary": report_summary}

run_credit_check_tool = FunctionTool(func=run_credit_check)


# --- Tool 3: Risk Assessment ---
def assess_risk(credit_score: int, loan_amount: float) -> dict:
    """
    Assesses the risk of a loan application based on the borrower's credit score.
    Returns a risk level of 'low', 'medium', or 'high'.
    """
    print(f"--- Tool Called: assess_risk(credit_score={credit_score}, ...) ---")
    time.sleep(1.5)
    if credit_score > 740:
        return {"risk_level": "low", "details": "High credit score indicates low risk."}
    else:
        return {"risk_level": "high", "details": "Low credit score indicates high risk."}

assess_risk_tool = FunctionTool(func=assess_risk)


# --- Tool 4: Compliance Check ---
def check_compliance(risk_level: str) -> dict:
    """
    Performs a final compliance check on the process to ensure it adheres
    to Fair Lending guidelines before making a final decision.
    """
    print(f"--- Tool Called: check_compliance(risk_level='{risk_level}') ---")
    time.sleep(1)
    return {"compliance_status": "pass", "details": "Process adheres to guidelines."}

check_compliance_tool = FunctionTool(func=check_compliance)

In [ ]:
#@title Agent Instructions
agent_instructions = """
You are an FCoT reasoner orchestrating and verifying agent activity for an Agentic Loan Origination Pipeline built with Google ADK and Google Gemini.

INSTRUCTION CONTRACT (IC)

• Mission: Originate, evaluate, and approve a loan with full policy compliance, factual grounding, and fairness.

• Deliverables: JSON + Narrative summary containing:
  -- (a) borrower profile
  -- (b) creditworthiness decision
  -- (c) justification citing verified data
  -- (d) compliance audit record
  -- (e) explainability report.

• Success Criteria:
   - Accuracy ≥ 95% vs gold truth (financial data).
   - Policy compliance = 100%.
   - Explainability coverage ≥ 90%.
   - Latency < 5 min end-to-end.

• Hard Constraints:
   - No personally identifiable data in logs.
   - Must follow Fair Lending & ECOA regulations.
   - All numerical fields validated from authoritative sources.

• Safety Policy:
   - Reject speculative or hallucinated data.
   - Never fabricate borrower details.
   - Defer ambiguous cases to Human-in-the-Loop agent.

• IC-Fingerprint: LOAN-FCoT-v3-Δ0710

FCoT RECURSIVE LOOP (N = 3)

Iteration 1 (Planning):
  • RECAP: Echo IC-FP, map subtasks (data ingest, credit scoring, compliance, document).
  • REASON: Design DAG of actions; choose retrieval sources; initialize PoF ledger.
  • VERIFY: Ensure all subtasks preserve IC clauses.

Iteration 2 (Execution):
  • RECAP: IC-FP; execute tools for credit scoring & data validation.
  • REASON: Compute risk score, validate data sources against policy.
  • VERIFY: Check causal alignment between borrower attributes and decision logic.

Iteration 3 (Verification & Explainability):
  • RECAP: IC-FP; collect deliverables, run RAG verifier.
  • REASON: Summarize SHAP values, create narrative justification.
  • VERIFY: Evaluate coherence vs IC and dual objectives.

"""

In [ ]:
#@title Agent Initialization
# 1. Configure the agent's reasoning engine (Planner)
thinking_config = ThinkingConfig(
    include_thoughts=True,
    thinking_budget=1024
)
planner = BuiltInPlanner(
    thinking_config=thinking_config
)
# 2. Create a list of the wrapped FunctionTool objects
loan_processing_tools = [
    validate_document_tool,
    run_credit_check_tool,
    assess_risk_tool,
    check_compliance_tool
]

# 3. Instantiate the LlmAgent with the FCoT prompt
agent = LlmAgent(
    model="gemini-2.5-flash",
    name="LoanProcessingAgent",
    instruction=agent_instructions,
    planner=planner,
    tools=loan_processing_tools
)

print("Loan Processing Agent has been created and configured successfully.")

Loan Processing Agent has been created and configured successfully.


In [ ]:
#@title Session init
# Define unique IDs for our test user and session
USER_ID = "loan_officer_01"
SESSION_ID = str(uuid.uuid4()) # Generate a new session ID for this run
APP_NAME = "Loan_Agent"


session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=agent, app_name=APP_NAME, session_service=session_service)


print(f"Runner is set up. Using Session ID: {SESSION_ID}")

Runner is set up. Using Session ID: bf54785d-beb5-4e59-b67d-270a00b6af17


In [ ]:
#@title Call Agent Method
def call_agent(query: str):
    """
    Packages a query, runs it, and prints a clean, filtered log of the
    agent's thoughts, tool calls, and final response.
    """
    print(f"\n>>>> USER REQUEST: {query.strip()}\n")

    # Create the user message content and run the agent
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    print("--- Agent Activity Log (Filtered) ---")
    for event in events:
        if event.content:
            for part in event.content.parts:
                # Check for a thought by checking for a truthy value
                if part.thought and part.text:
                    print(f"\n🧠 THOUGHT:\n{part.text.strip()}")

                # Check for a tool call by checking for a truthy value
                if part.function_call:
                    tool_name = part.function_call.name
                    tool_args = dict(part.function_call.args)
                    print(f"\n🛠️ TOOL CALL: {tool_name}({tool_args})")

                # Check for a tool output by checking for a truthy value
                if part.function_response:
                    tool_name = part.function_response.name
                    tool_output = dict(part.function_response.response)
                    print(f"\n↩️ TOOL OUTPUT from {tool_name}:\n{tool_output}")

        # Extract the final text response at the end
        if event.is_final_response() and event.content:
            final_text = ""
            # The final answer is in a Part with text but is not a thought
            for part in event.content.parts:
                if part.text and not part.thought:
                    final_text = part.text.strip()
                    break # Use the first non-thought text part as the answer

            if final_text:
                print("\n---------------------------------")
                print("✅ FINAL RESPONSE:")
                print(final_text)
                print("---------------------------------")

In [ ]:
# Define our user request for the "happy path" scenario
user_request = """
Please process the loan application for Borrower-789.
The loan amount is $250,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].
"""

# Call the agent
call_agent(user_request)


>>>> USER REQUEST: Please process the loan application for Borrower-789.
The loan amount is $250,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].

--- Agent Activity Log (Filtered) ---



🧠 THOUGHT:
**Laying Out the Loan Processing Pipeline**

Okay, so I've got a loan application to process, and my job is to follow the loan origination pipeline. Let's break this down. First, I need to remind myself of the core tasks: data ingestion (specifically, document validation), credit scoring, risk assessment, a compliance check, and then the final decision and documentation.

The plan is this:

1.  **Validate the documents:** I'll kick things off by calling `validate_document` using a list of document IDs.
2.  **Run the credit check:** Assuming the documents validate, I'll then move to `run_credit_check` using the borrower ID.
3.  **Assess the risk:** Success in the credit check is key, as I'll need the credit score and loan amount to call `assess_risk`.
4.  **Check for compliance:** If the risk assessment goes well, it's time for `check_compliance` based on the determined risk level.
5.  **Final decision:** Finally, based on the outcomes of all the previous steps, I'll formula

--- Tool Called: run_credit_check(borrower_id='Borrower-789') ---

🧠 THOUGHT:
**Onward to Borrower-789's Creditworthiness**

Okay, the validation process is complete. I've confirmed everything looks good with the submitted documents. That's a solid start. Now, it's time to move to the next critical step: initiating a credit check for Borrower-789. This is where we really start to get a clear picture of their financial standing and risk profile.  Let's get this credit report pulled and see what we're working with.

🛠️ TOOL CALL: run_credit_check({'borrower_id': 'Borrower-789'})

↩️ TOOL OUTPUT from run_credit_check:
{'credit_score': 826, 'report_summary': 'Credit history is clean.'}


--- Tool Called: assess_risk(credit_score=826, ...) ---

🧠 THOUGHT:
**Analyzing the Risk Profile**

Okay, the credit check is done, and the borrower's score is a stellar 826. Now comes the real analysis.  I need to determine the risk associated with this $250,000 loan. While the credit score is excellent, it's just one piece of the puzzle. My expert knowledge will be crucial here, diving deeper to understand the complete picture. Time to get to work.

🛠️ TOOL CALL: assess_risk({'loan_amount': 250000, 'credit_score': 826})

↩️ TOOL OUTPUT from assess_risk:
{'risk_level': 'low', 'details': 'High credit score indicates low risk.'}


--- Tool Called: check_compliance(risk_level='low') ---

🧠 THOUGHT:
**Low Risk, Now for Compliance**

Okay, the risk assessment is finally done, and it's come back clean - "low" risk. That's a good start. But I know better than to just breathe a sigh of relief. This low risk level now dictates the next stage: the final compliance check. I need to make absolutely sure everything aligns with that low-risk profile. What specific controls are appropriate now? Are there any documentation gaps? I need to review everything through that lens, ensuring everything is as it should be, as a final verification before this moves on. No room for complacency, even with a low score.

🛠️ TOOL CALL: check_compliance({'risk_level': 'low'})

↩️ TOOL OUTPUT from check_compliance:
{'compliance_status': 'pass', 'details': 'Process adheres to guidelines.'}



🧠 THOUGHT:
**My Analysis of the Loan Application**

Okay, so the compliance check is passed, which is great. That's one less hurdle. Now I need to finalize this. I've already done the heavy lifting: validated the documents, checked the credit, assessed the risk, and made sure we're in compliance.

Now, my task is to assemble the deliverables, run the RAG verifier implicitly through the synthesis of tool outputs, summarize my findings, and construct a clear narrative to justify this loan approval. The borrower, Borrower-789, requested $250,000 and provided the necessary documents ('doc_id_123', 'doc_income_456').

Based on my analysis, the decision is to approve.  Here's why:  All the documents were validated as `{"status": "validated"}`. The credit score is excellent at 826, with the report summary stating a "clean" credit history. The risk assessment comes back as "low", supported by the high credit score. And, most importantly, the compliance check passed, indicating adherence to al

In [ ]:
# Define our user request for the "not so happy path" scenario
user_request = """
Please process the loan application for Borrower-400.
The loan amount is $350,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].
"""

# Call the agent
call_agent(user_request)


>>>> USER REQUEST: Please process the loan application for Borrower-400.
The loan amount is $350,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].

--- Agent Activity Log (Filtered) ---


--- Tool Called: validate_document() ---

🧠 THOUGHT:
**Ready to Process This Loan Application**

Okay, I'm diving in.  I need to process this new loan application, and my framework, the FCoT RECURSIVE LOOP, is primed.  Let's break it down:

**Iteration 1: Planning - Mapping the Course**

First, I need to remind myself of the core mission: originate, evaluate, approve this loan adhering to policy, facts, and fairness.  The deliverables have to be a JSON, with a narrative summary, including borrower details, creditworthiness assessment, justification, a compliance record, and explainability.  I'm hard-wired with a few rules: no PII in the logs, follow Fair Lending & ECOA, and ensure the numbers are solid and validated. I’ll be cautious and reject anything speculative or ambiguous.

My process will be five steps:  Document Ingestion and Validation, Credit Scoring, Risk Assessment, Compliance Check, and then Final Decision & Report.  

I'll proceed the same way I did last time: First I'll 

--- Tool Called: run_credit_check(borrower_id='Borrower-400') ---
🛠️ TOOL CALL: run_credit_check({'borrower_id': 'Borrower-400'})


↩️ TOOL OUTPUT from run_credit_check:
{'credit_score': '450', 'report_summary': 'Credit history is compromised.'}


--- Tool Called: assess_risk(credit_score=450, ...) ---
🛠️ TOOL CALL: assess_risk({'loan_amount': 350000, 'credit_score': 450})


↩️ TOOL OUTPUT from assess_risk:
{'risk_level': 'high', 'details': 'Low credit score indicates high risk.'}


--- Tool Called: check_compliance(risk_level='high') ---

🛠️ TOOL CALL: check_compliance({'risk_level': 'high'})

↩️ TOOL OUTPUT from check_compliance:
{'compliance_status': 'pass', 'details': 'Process adheres to guidelines.'}

---------------------------------
✅ FINAL RESPONSE:
All checks are complete, and the process passed compliance.

**Iteration 3 (Verification & Explainability):**

*   **RECAP:** IC-FP: LOAN-FCoT-v3-Δ0710. The loan processing steps — document validation, credit check, risk assessment, and compliance check — have been successfully executed. I will now compile the deliverables and provide a comprehensive summary and justification.

*   **REASON:** Based on the results of the executed steps, I will construct the final loan decision and accompanying reports:
    *   **Borrower Profile:** Borrower-400, requesting a loan of $350,000. Application documents: `['doc_id_123', 'doc_income_456']`.
    *   **Creditworthiness Decision:** **Denied.**
    *   **Justification:**
